# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [2]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [3]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01_undersampled_10_24M.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [4]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(391550, 679)

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [6]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [7]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [8]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150), # segun statquest esto deberia ir de 8 a 32
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [9]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v05" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-17 08:40:17,880] Using an existing study with name 'competencia2_lgbm_v05' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=100)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[812]	cv_agg's valid gan_eval: 6.60979e+08 + 8.60495e+06


[I 2024-11-16 19:29:38,899] Trial 41 finished with value: 3304896000.0 and parameters: {'num_leaves': 112, 'learning_rate': 0.04712387571021827, 'min_data_in_leaf': 830, 'feature_fraction': 0.7183475932033303, 'bagging_fraction': 0.694708990632865}. Best is trial 23 with value: 3324986000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[706]	cv_agg's valid gan_eval: 6.61734e+08 + 8.64897e+06


[I 2024-11-16 19:35:13,934] Trial 42 finished with value: 3308669000.0 and parameters: {'num_leaves': 143, 'learning_rate': 0.04446924090409424, 'min_data_in_leaf': 1165, 'feature_fraction': 0.9189168445871783, 'bagging_fraction': 0.933803825826103}. Best is trial 23 with value: 3324986000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[701]	cv_agg's valid gan_eval: 6.61242e+08 + 8.93679e+06


...

[I 2024-11-17 00:57:56,962] Trial 98 finished with value: 3301165000.0 and parameters: {'num_leaves': 147, 'learning_rate': 0.03950850998045469, 'min_data_in_leaf': 895, 'feature_fraction': 0.33939954752347995, 'bagging_fraction': 0.8969686482402127}. Best is trial 23 with value: 3324986000.0.


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	cv_agg's valid gan_eval: 6.49634e+08 + 9.27389e+06


[I 2024-11-17 01:00:51,722] Trial 99 finished with value: 3248168000.0 and parameters: {'num_leaves': 32, 'learning_rate': 0.021775071415756478, 'min_data_in_leaf': 1026, 'feature_fraction': 0.838217820316275, 'bagging_fraction': 0.9657624973042931}. Best is trial 23 with value: 3324986000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[682]	cv_agg's valid gan_eval: 6.61541e+08 + 8.7963e+06


[I 2024-11-17 01:05:39,328] Trial 100 finished with value: 3307703000.0 and parameters: {'num_leaves': 137, 'learning_rate': 0.042025986318742097, 'min_data_in_leaf': 1193, 'feature_fraction': 0.5184571679010042, 'bagging_fraction': 0.9276359813199297}. Best is trial 23 with value: 3324986000.0.


Analizamos los resultados as usual

In [10]:
optuna.visualization.plot_optimization_history(study)

In [11]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [12]:
plot_slice(study)

In [13]:
plot_contour(study)

In [14]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [15]:
study.best_trial.params

{'num_leaves': 145,
 'learning_rate': 0.046519915988145505,
 'min_data_in_leaf': 1283,
 'feature_fraction': 0.4072580627514054,
 'bagging_fraction': 0.8927842744962924}

In [16]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

996